In [132]:
import os
import pandas as pd
import glob
import re

In [133]:
class Utils():
    root_directory = os.chdir('c:/Users/iZiPC.by/notebooks/parser/')# input path
    list_of_files = glob.glob('**/*.txt', recursive=True)
    invalid_parsing = []
    trash = ['Волковыск-1.txt','Волковыск.txt','Вороново-1.txt',
         'Вороново.txt', 'Черновик-1.txt', 'Черновик.txt']
    for file in list_of_files:
        try:
             with open(file, encoding='utf-8') as f:
                    num = f.readline()
                    file_ = f.read()
                    if num and file_:
                        pass
        except:
            invalid_parsing.append(file)
    for i in invalid_parsing:
        list_of_files.remove(i)
    for i in trash: 
        list_of_files.remove(i)


In [134]:
list_of_files = Utils.list_of_files

In [135]:
plt_ = []
wbc_ = []
hgb_ = []
crea_ = []
ldh_ = []
sex_ = []
crp_ = []
tr = []
bir = []
ad = []
dis = []
ID_ = []
dgs_ = []

In [136]:
def ID(num):
  """returns ID of the patient"""
  for i in num.split():
    for char in i.split():
        if char.isnumeric():
            try:
                return int(char)
            except:
                return 'None'


In [137]:
def treatment(file_):
  """ returns srting of remedies used for the patient """
  for line in file_.split('\n'):
    if re.search('лечение', line):  # Does the same thing as "if 'hello' in line:"
        return(line.replace('Проведенное лечение:', '').replace('ЛФК', '').replace('ФТЛ', ''))

In [138]:
def born_adm_disch(file_):
    """ finds all nessesary dates in the epicrisis"""
    pattern = re.compile("(\d{2}).(\d{2}).(\d{4})") # check all dates
    birthday, admission = pattern.findall(file_)[:2]
    discharging = pattern.findall(file_)[-1]
    try:
        return ':'.join(birthday), ':'.join(admission), ':'.join(discharging)
    except:
        return 'None'

In [139]:
pattern_diagnosis = ['Диагноз:','Диагноз клинический:','Диагноз', 'Д-з:', "Диагноз заключительный:"]
def diagnosis_dirty(file_):
  int_=0
  for i in file_.split():
    int_ += 1
    if i in pattern_diagnosis:
      return file_.split()[int_+1:int_+30]

In [140]:
def crp(file_):
  """returns all crp values"""
  pattern = re.sub(r'[ЦСC]РБ', 'С-реактивныйбелок', file_)
  pattern_1 = re.compile(r'(?:\w\Dреактивныйбелок|\w\Dреактивныйбелокдо)(\d*.\d+|\d+)')
  pattern_2 = pattern_1.findall(''.join(pattern.split()))
  try:
    return pattern_2
  except:
    return 'None'

In [141]:
def gender(file_):
  patt1 = re.compile(r'(?<=Ф.И.О:)[А-Я]\w{,20}[А-Я]\w{,20}(вич)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return 'male'
    else:
      return 'female'
  except:
      pass

In [142]:
def ldh(file_):
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wактатдегидрогеназ\w', 'лдг', file_)
  patt1 = re.compile(r'(?<=лдг)\d{,4}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass

In [143]:
def cre(file_):
  """Returns all creatinine values"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'(\wреатини\w|креатин|креат)(?![а-яА-Я,])', 'cre', file_)
  patt1 = re.compile(r'(?<=cre)\d{,3}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass

In [144]:
def hgb(file_):
  """returns list of str with level of hemoglobin"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  patt = re.compile(r'гемоглобин|гемогл|гб|гем')
  patt1 = re.sub(patt, r'hgb', file_)
  patt2 = re.compile(r'(?<=hgb)\d{2,3}')
  all_hgb = patt2.findall(''.join(patt1.split()))
  try:
    if all_hgb:
      return all_hgb
    else:
      return 'None'
  except:
    pass

In [145]:
def wbc(file_):
  """returns list of str with level of wbc"""
  file_ = file_.lower()
  patt1 = re.sub(r'[():]', '', file_)
  patt1 = re.sub(r'10(\*|[еe])9', '', patt1)
  patt1 = re.sub(r'\Dбщийанализкрови', 'оак', patt1) 
  patt2 = re.sub(r'(?<=оак)\d\d.\d\d.\d{2,4}|(?<=оак)\d\d.\d\d', '', patt1)
  patt3 = re.compile(r'(?<=оак|wbc)(?:л|лейкоцит\w)(\d*.\d+|\d+)')
  all_wbc = patt3.findall(''.join(patt2.split()))
  try:
    if all_wbc:
      return all_wbc
    else:
      return 'None'
  except:
    pass

In [146]:
def plt(file_):
  """returns list of str with level of plt"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  patt2 = re.compile(r'(?:\wромбоцит\w|тр)')
  patt1 = re.sub(patt2, '', file_)
  patt1 = ''.join(patt1.split())
  patt2 = re.compile(r'(?<=plt)\d{3}')
  all_plt = patt2.findall(patt1)
  try:
    if all_plt:
      return all_plt
    else:
      return 'None'
  except:
    pass

In [147]:

for file in Utils.list_of_files:
    try:
         with open(file, encoding='utf-8') as f:
                num = f.readline()
                file_ = f.read()
                ID_.append(ID((num)))
                tr.append(treatment((file_)))
                bir.append(born_adm_disch(file_)[0])
                ad.append(born_adm_disch(file_)[1])
                dis.append(born_adm_disch(file_)[2])
                dgs_.append(diagnosis_dirty((file_)))
                crp_.append(crp(''.join(file_.split())))
                sex_.append(gender(''.join(file_.split())))
                ldh_.append(ldh(file_))
                crea_.append(cre(file_))
                hgb_.append(hgb(file_))
                wbc_.append(wbc(file_))
                plt_.append(plt(file_))
    except:
        print(file)

In [149]:
data_dct = {'Platlets':plt_, 'Leucocytes': wbc_, 'Hemoglobin': hgb_,
           'Creatinine': crea_, 'LDG':ldh_, 'Gender':sex_, 'CRP':crp_,
                                  'Treatment':tr, 'Birthday': bir, 'Admission':ad,
                                  'Discharge':dis, 'ID':ID_, 'Diagnosis': dgs_}

In [150]:
dataFrame = pd.DataFrame(data_dct)


In [151]:
dataFrame.head()

,Platlets,Leucocytes,Hemoglobin,Creatinine,LDG,Gender,CRP,Treatment,Birthday,Admission,Discharge,ID,Diagnosis
0,[320],[5.81],[151],None,None,male,[],"левофлоксацин . амброксол , парацетамол , иб...",12:09:1988,23:09:2019,02:10:2019,5616.0,"[Внегоспитальная, полисегментарная, пневмония,..."
1,[444],[7.15],[138],[86],None,female,[],"триксоцеф. азитромицин, амброксол,",22:10:1998,18:01:2018,29:01:2018,430.0,"[J18, Пневмония, без, уточнения, возбудителя/,..."
2,"[148, 371, 159]","[8.2, 5.76, 7.79]","[146, 164, 165]",None,None,female,"[176,1, 5.45]","цефтриаксон, амброксол, плевилокс,",10:05:1963,17:02:2020,13:03:2020,1103.0,"[J18, Пневмония, без, уточнения, возбудителя/,..."
3,None,None,None,None,None,female,[],None,03:07:1940,18:05:2018,01:06:2018,3338.0,None
4,[236],[10.42],[146],None,None,male,[6],"цефтриаксон . амброкосл . аспикард , аторв...",07:09:1957,22:01:2019,30:01:2019,498.0,"[Внегоспитальная, пневмония, в, нижней, доле, ..."
